In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="your roboflow key")
project = rf.workspace("roboflow-gw7yv").project("self-driving-car")
dataset = project.version(3).download("mt-yolov6")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 26.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.0.76
    Uninstalling opencv-python-headless-4.8.0.76:
      Successfully uninstalled opencv-python-headless-4.8.0.76
  Attempting uninstall: idna
    Found

A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Self-Driving-Car-3 in mt-yolov6:: 100%|██████████| 59607/59607 [00:07<00:00, 7551.47it/s] 


In [4]:
%ls

Self-Driving-Car-3/


In [5]:
import os
import math


def get_weighted_list(images_dir, weights=(0.9, 0.1, 0.0)):
    img_arr = os.listdir(images_dir)
    img_len = len(img_arr)

    train_i = math.floor(img_len * weights[0])
    valid_i = math.floor(img_len * weights[1])
    test_i = math.floor(img_len * weights[2])

    print('Used %d out of %d images' % (train_i + valid_i + test_i, img_len))

    return [
        img_arr[:train_i],
        img_arr[train_i:train_i + valid_i],
        img_arr[train_i + valid_i:train_i + valid_i + test_i]
    ]


def create_dir(dataset_dir, category_dir, subset_dir):
    for category in category_dir:
        for subset in subset_dir:
            computed_path = os.path.join(dataset_dir, category, subset)
            if not os.path.isdir(computed_path):
                os.makedirs(computed_path)
                print('Directory does not exist, creating directory: %s' % computed_path)


dataset = 'Self-Driving-Car-3'
images_d = 'images/export'
labels_d = 'labels/export'
category_d = ['images', 'labels']
subset_d = ['train', 'valid', 'test']


def main():
    # Create directories if they don't exist
    create_dir(dataset, category_d, subset_d)

    # Create weighted list of images by providing its directory
    weighted_list = get_weighted_list(os.path.join(dataset, images_d), (0.6, 0.2, 0.2))

    for i, category in enumerate(weighted_list):
        for file in category:
            org_img = os.path.join(dataset, images_d, file)
            org_txt = os.path.join(dataset, labels_d, file)
            org_txt = os.path.splitext(org_txt)[0]+'.txt'

            image_path = os.path.join(dataset, category_d[0], subset_d[i], file)
            txt_path = os.path.join(dataset, category_d[1], subset_d[i], file)
            txt_path = os.path.splitext(txt_path)[0]+'.txt'

            os.popen('cp %s %s' % (org_img, image_path))
            os.popen('cp %s %s' % (org_txt, txt_path))


if __name__ == "__main__":
    main()


Directory does not exist, creating directory: Self-Driving-Car-3/images/train
Directory does not exist, creating directory: Self-Driving-Car-3/images/valid
Directory does not exist, creating directory: Self-Driving-Car-3/images/test
Directory does not exist, creating directory: Self-Driving-Car-3/labels/train
Directory does not exist, creating directory: Self-Driving-Car-3/labels/valid
Directory does not exist, creating directory: Self-Driving-Car-3/labels/test
Used 29800 out of 29800 images


In [7]:
!git clone https://github.com/meituan/YOLOv6
%cd YOLOv6

Cloning into 'YOLOv6'...
remote: Enumerating objects: 3785, done.
remote: Counting objects: 100% (1658/1658), done.
remote: Compressing objects: 100% (291/291), done.
remote: Total 3785 (delta 1466), reused 1369 (delta 1367), pack-reused 2127
Receiving objects: 100% (3785/3785), 47.11 MiB | 37.33 MiB/s, done.
Resolving deltas: 100% (2304/2304), done.
/kaggle/working/YOLOv6/YOLOv6


In [8]:
!pip install addict
!pip install pycocotools

In [15]:
!python tools/train.py --batch 32 --conf configs/yolov6n.py --epochs 70 --img-size 416 --data /kaggle/input/self-driving/data.yaml --device 0,1

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
0 image(s) corrupted: 100%|█████████████| 17880/17880 [00:03<00:00, 4963.11it/s]
17880 label(s) found, 0 label(s) missing, 2084 label(s) empty, 0 invalid label f
0 image(s) corrupted: 100%|███████████████| 5960/5960 [00:02<00:00, 2203.53it/s]
5960 label(s) found, 0 label(s) missing, 745 label(s) empty, 0 invalid label fil
100%|████████████████████████████████████| 5960/5960 [00:00<00:00, 11363.35it/s]
      0/69       0.02     1.315         0     1.142: 100%|██████████| 559/559 [0
      1/69       0.02    0.9024         0    0.9618: 100%|██████████| 559/559 [0
      2/69    0.01999    0.7637         0    0.9102: 100%|██████████| 559/559 [0
      3/69    0.01996    0.6911         0    0.8684: 100%|██████████| 559/559 [0
      4/69    0.01991